In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.ai.ml.dsl import pipeline
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /home/daniel/repos/aml2/config.json


In [6]:
from azure.ai.ml import command, Input, Output

#import Data object to register a dataset
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

In [7]:
component1 = command(
    name="component1",
    yaml_file="component1.yaml",
    version="1",
    display_name="component1_job",
    inputs={"data": Input(type="uri_folder")},
    outputs={"output": Output(type="uri_folder")},
    code = "./components",
    command = "python component1.py --data azureml:coco128:1 --output azureml:test_output:1",
    compute="defaultclustersbxdondorp",
    environment="arureml:dockerenv:1"
)

In [8]:
@pipeline(compute_target="defaultclustersbxdondorp")
def my_pipeline(data):
    output = component1(data=data)
    return output

In [9]:
ml_client.create_or_update(my_pipeline(data="azureml:coco128:1"))

HttpResponseError: (UserError) Error occurred when loading YAML file rootNode, details: Invalid environment id: No definition exists for environment arureml version dockerenv:1. See https://aka.ms/azureml/environment/no-definition-exists for troubleshooting.
Code: UserError
Message: Error occurred when loading YAML file rootNode, details: Invalid environment id: No definition exists for environment arureml version dockerenv:1. See https://aka.ms/azureml/environment/no-definition-exists for troubleshooting.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "414cb763194e4194e416cc26b6af9af2",
        "request": "2d50764c4911ce5f"
    }
}Type: Environment
Info: {
    "value": "westeurope"
}Type: Location
Info: {
    "value": "westeurope"
}Type: Time
Info: {
    "value": "2025-02-19T12:35:09.7393918+00:00"
}Type: InnerError
Info: {
    "value": {
        "code": "BadArgument",
        "innerEr

In [88]:
from azure.ai.ml import load_component

In [90]:
component1 = load_component("./components/component1.yaml")
component1 = ml_client.create_or_update(component1)

In [57]:
my_data = Data( 
    path="test_folder",  
    type=AssetTypes.URI_FOLDER,    
    name="testfolder",
    description="test folder"
)  

In [59]:
!echo "There is a file in the folder" > test_folder/test.txt

In [60]:
ml_client.data.create_or_update(my_data)

Uploading test_folder (0.0 MBs): 100%|██████████| 30/30 [00:00<00:00, 1256.72it/s]




Data({'path': 'azureml://subscriptions/11f51dee-57cd-4d47-b542-8e244706e163/resourcegroups/sbx-dondorp/workspaces/amlsbxdondorp/datastores/workspaceblobstore/paths/LocalUpload/86a524c5b5095525dac32efac822a261/test_folder/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'testfolder', 'description': 'test folder', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/11f51dee-57cd-4d47-b542-8e244706e163/resourceGroups/sbx-dondorp/providers/Microsoft.MachineLearningServices/workspaces/amlsbxdondorp/data/testfolder/versions/1', 'Resource__source_path': '', 'base_path': '/home/daniel/repos/aml/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fc99ef9ae30>, 'serialize': <msrest.serialization.Serializer object at 0x7fc99ef9b9d0>, 'version': '1', 'latest_version': None, 'datastore': Non

In [105]:
outfolder = Output(type="uri_folder", path="azureml:testfolder")
infolder = Input(type="uri_folder", path="azureml:testfolder")

job1 = command(
    command="""
        echo 'hello world, this is at test file' >> azureml:testfolder/test.txt
    """,
    environment="azureml:dockerenv:1",
    compute="defaultclustersbxdondorp",
    display_name="hello_world_pipeline",
    experiment_name="hello_world_pipeline",
    outputs = dict(outfolder = outfolder)
    )

job2 = command(
    inputs=dict(infolder = infolder),
    command = f"cat {infolder.path}/test.txt",
    environment="azureml:dockerenv:1",
    compute="defaultclustersbxdondorp",
    display_name="hello_world_pipeline_pt2",
    experiment_name="hello_world_pipeline_pt2"
)



In [106]:
infolder.path

'azureml:testfolder'

In [107]:
ml_client.jobs.create_or_update(job1)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
hello_world_pipeline,kind_net_l6rb8hdg1k,command,Starting,Link to Azure Machine Learning studio


In [108]:
ml_client.jobs.create_or_update(job2)

MlException: 
[37m
[30m
1) At least one required parameter is missing[39m[39m

Details: 

[31m(x) Failed to extract version when parsing asset azureml:testfolder of type data as arm id. Version must be provided.[39m

Resolutions: 
1) Ensure all parameters required by the Asset schema are specified.
If using the CLI, you can also check the full log in debug mode for more details by adding --debug to the end of your command

Additional Resources: The easiest way to author a yaml specification file is using IntelliSense and auto-completion Azure ML VS code extension provides: [36mhttps://code.visualstudio.com/docs/datascience/azure-machine-learning.[39m To set up VS Code, visit [36mhttps://docs.microsoft.com/azure/machine-learning/how-to-setup-vs-code[39m


In [109]:
@pipeline
def my_pipeline():
    job1()
    job2()

ml_client.jobs.create_or_update(my_pipeline())

MlException: 
[37m
[30m
1) At least one required parameter is missing[39m[39m

Details: 

[31m(x) Failed to extract version when parsing asset azureml:testfolder of type data as arm id. Version must be provided.[39m

Resolutions: 
1) Ensure all parameters required by the Asset schema are specified.
If using the CLI, you can also check the full log in debug mode for more details by adding --debug to the end of your command

Additional Resources: The easiest way to author a yaml specification file is using IntelliSense and auto-completion Azure ML VS code extension provides: [36mhttps://code.visualstudio.com/docs/datascience/azure-machine-learning.[39m To set up VS Code, visit [36mhttps://docs.microsoft.com/azure/machine-learning/how-to-setup-vs-code[39m
